In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Defesas difíceis

#### O que é uma defesa difícil?

Defesas difíceis são defesas feitas pelo goleiro e que possuem alto grau de dificuldade. 

Uma defesa difiícil na maioria das vezes se origina de um chute a gol, porém também pode se originar de um passe com alta possibilidade de gol, que acontece geralmente quando o goleiro saí do gol para interceptar um escanteio. 

Essas defesas também resultam, principalmente, em rebotes, escanteios ou posse de bola para o time do goleiro.

#### Análise geral da coluna

A coluna de defesas difíceis possui uma alta quantidade de dados ausentes, sendo esses equivalente a aproximadamente 78% dos dados totais. Por isso, é preciso achar um forma de preencher esses dados.

In [ ]:
df = pd.read_csv('campeonato.csv')

defesas = df[['Defesas difíceis 1', 'Defesas difíceis 2']]
print(defesas.describe(),'\n')
print("Porcentagem de dados nulos =", round((1-len(defesas.dropna())/len(df))*100),"%")

Para analisar a relevância da coluna, utilizamos uma tabela de correlação, que mostra a influência das colunas Defesas difíceis nas demais.

In [ ]:
#CORRELAÇÃO DEFESAS DIFÍCEIS 1
correlacoes = df.corr(numeric_only=True)["Defesas difíceis 1"].drop("Defesas difíceis 1")

plt.figure(figsize=(8, 5))
correlacoes.sort_values().plot(kind='barh', color='skyblue')
plt.title("Correlação com Defesas Difíceis 1")
plt.xlabel("Coeficiente de Correlação")
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#CORRELAÇÃO DEFESAS DIFÍCEIS 2
correlacoes = df.corr(numeric_only=True)["Defesas difíceis 2"].drop("Defesas difíceis 2")

plt.figure(figsize=(8, 5))
correlacoes.sort_values().plot(kind='barh', color='skyblue')
plt.title("Correlação com Defesas Difíceis 2")
plt.xlabel("Coeficiente de Correlação")
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Através dos dois gráficos, podemos perceber uma altíssima correlação de Defesas difíceis com Chutes a gol. Isso acontece, pois a grande maioria dos chutes a gol são convertidos em defesas difíceis ou em gols, relação que pode ser exemplificada na seguinte tabela, em que a soma de Defesas difíceis X (1 ou 2) com Gols Y, é muito próximo de Chutes a gol Y.

In [ ]:
teste = df[['Defesas difíceis 2', 'Gols 1', 'Chutes a gol 1', 'Defesas difíceis 1', 'Gols 2', 'Chutes a gol 2']].dropna()
teste.describe()

Portanto, faz sentido tentar preencher a coluna de Defesas difíceis com a diferença entre Chutes a gol e Gols de um time.

In [ ]:
# Calcula a quantidade de defesas de defesas a partir da diferença entre Chutes a gol e Gols
teste['Defesas 1'] = teste['Chutes a gol 2'] - teste['Gols 2']

#Caso Defesas receba um número negativo, altera ele para 0
teste.loc[teste['Defesas 1'] < 0, 'Defesas 1'] = 0

#Imprime a diferença entre o resulutado e o valor esperado
teste['Diferença1'] = teste['Defesas difíceis 1'] - teste['Defesas 1']
print(teste['Diferença1'].value_counts().sort_index())

#Repete o processo para Defesas 2
teste['Defesas 2'] = teste['Chutes a gol 1'] - teste['Gols 1']
teste.loc[teste['Defesas 2'] < 0, 'Defesas 2'] = 0
teste['Diferença2'] = teste['Defesas difíceis 2'] - teste['Defesas 2']
print(teste['Diferença2'].value_counts().sort_index())

Esse método de preencher os dados faltantes pareceu bastante constante, possuindo um taxa de acerto de aproximadamente 88%, e chegando a 99% quando considerada uma margem de até 1 defesa difícil. Além disso, como Chutes a gol e Gols têm muito menos dados faltantes, a maioria das colunas da nossa base de dados poderia ser preenchida comesse método.

OBS: Existem alguns casos em que a quantidade de chutes a gol é menor do que a quantidade de defesas difíceis. Inicialmente, achamos que era uma incoerência nos dados, porém pesquisando mais a fundo, descobrimos que uma defesas difícil nem sempre é feita em um chute a gol, por isso decidimos manter essas linhas para nosso modelo.

In [ ]:
#Imprime a quantidade de linhas com mais defesas difíceis do que chutes a gol
len(df[(df['Chutes a gol 1']<df['Defesas difíceis 2'])|(df['Chutes a gol 2']<df['Defesas difíceis 1'])])

#### Conlusão

Como o método de usar a diferença entre Chutes a gol e Gols pareceu bem constante, nós optamos por utilizá-lo para preencher os dados faltantes. Isso fez com  que a porcentagem de dados nulos fosse para 5%, a mesma porcentagem de dados nulos de Chutes a gol.

In [ ]:
mascara1 = df['Defesas difíceis 1'].isna() & df['Chutes a gol 2'].notna()
df.loc[mascara1, 'Defesas difíceis 1'] = df.loc[mascara1, 'Chutes a gol 2'] - df.loc[mascara1, 'Gols 2']

mascara2 = df['Defesas difíceis 2'].isna() & df['Chutes a gol 1'].notna()
df.loc[mascara2, 'Defesas difíceis 2'] = df.loc[mascara2, 'Chutes a gol 1'] - df.loc[mascara2, 'Gols 1']

In [ ]:
print("Porcentagem de dados nulos =", round((1-len(df[['Defesas difíceis 1', 'Defesas difíceis 2']].dropna())/len(df))*100),"%")

In [ ]:
df.to_csv('campeonato.csv', index=False)